# Examples of utils provided by ReGraph's neo4j module

In [1]:
import networkx as nx

from regraph import Rule, plot_rule
from regraph.neo4j.graphs import Neo4jGraph
from regraph.neo4j.cypher_utils import *

## Initializing Neo4j graph

0. When installing neo4j you will be asked to choose login/password for you dbs (here its "neo4j"/"admin"), if you choose other -- change in the cell below.
1. To start neo4j server run `sudo service neo4j start`
2. Check status by running `sudo service neo4j status`. Here you can check the _bolt_ port, change the cell below if different from 7687
3. You can query the db by using the neo4j browser, the address can be found also in the result of 'status', e.g. "Remote interface available at http://localhost:7474/".

In [2]:
# initialize the neo4j driver, wrappped into Neo4jGraph object
g = Neo4jGraph(uri="bolt://localhost:7687", user="neo4j", password="admin")

In [3]:
# here we clear the db
res = g.clear()

In [4]:
nodes = [
    ("a", {"name": "Jack", "age": 23, "hobby": {"hiking", "music"}, "weight": 75}), 
    ("b", {"name": "Bob", "age": 24, "hobby": {"sport", "music"}, "height": 178}),
    "c", 
    ("d", {"name": "Paul"}), "e", "f"
]
edges = [
    ("a", "b", {"type": {"friends", "colleagues"}}), 
    ("d", "b", {"type": "enemies"}), 
    ("a", "c"),
    ("d", "a", {"type": "friends"}),
    ("e", "a"), 
    ("f", "d")
]
g.add_nodes_from(nodes)
g.add_edges_from(edges)

In [5]:
print(get_node("a"))
print(g.get_node("a"))

MATCH (n:node { id : 'a' })
RETURN n

{'name': ['Jack'], 'weight': [75], 'id': 'a', 'age': [23], 'hobby': ['hiking', 'music']}


## Primitive operations on Neo4j graph

### Basic operations

In [6]:
g.add_node("x", {"name": "Roberto"})

In [7]:
g.add_edge("x", "c", {"type": {"friends", "colleagues"}, "since": 1993})

In [8]:
g.nodes()

['a', 'b', 'c', 'd', 'e', 'f', 'x']

In [9]:
g.edges()

[('d', 'a'),
 ('e', 'a'),
 ('a', 'b'),
 ('d', 'b'),
 ('x', 'c'),
 ('a', 'c'),
 ('f', 'd')]

In [10]:
g.get_node('a')

{'age': [23],
 'hobby': ['hiking', 'music'],
 'id': 'a',
 'name': ['Jack'],
 'weight': [75]}

In [11]:
g.get_edge('a', 'b')

{'type': ['colleagues', 'friends']}

### Cloning

By default if we clone a node without specifying the new id for it, the id is automatically derived from the original one (however it takes some additional time to generate it, and some naming collisions may happen in the future when using clone operation inside of complex queries)

In [12]:
clone1_res = g.clone_node('a')
# print("Created clone by the name: ", clone1_res)
# print("Properties of the resulting node: ")
# print(g.get_node(clone1_res))
# print("Properties of incident edges are also cloned, e.g: ")
# print(g.get_edge(clone1_res, 'b'))

MATCH (x:node { id : 'a' })
WITH [] as sucIgnore, [] as predIgnore, x 
// match successors and out-edges of a node to be cloned
OPTIONAL MATCH (x)-[out_edge:edge]->(suc) 
WHERE NOT suc.id IS NULL AND NOT suc.id IN sucIgnore
WITH collect({neighbor: suc, edge: out_edge}) as suc_maps, predIgnore, x 
// match predecessors and in-edges of a node to be cloned
OPTIONAL MATCH (pred)-[in_edge:edge]->(x) 
WHERE NOT pred.id IS NULL AND NOT pred.id IN predIgnore
WITH collect({neighbor: pred, edge: in_edge}) as pred_maps, x, suc_maps 
// search for a node with the same id as the clone id
OPTIONAL MATCH (same_id_node:node { id : 'a'}) 
WITH same_id_node,  CASE WHEN same_id_node IS NOT NULL THEN (coalesce(same_id_node.count, 0) + 1) ELSE 0 END AS same_id_node_new_count, x, suc_maps, pred_maps
// generate new id if the same id node was found
// and filter edges which will be removed 
WITH same_id_node, same_id_node_new_count, 'a' + CASE WHEN same_id_node_new_count <> 0 THEN toString(same_id_node_new_c

We can specify the parameter `ignore_naming=True` to avoid automatic id derivation. In this case cloning uses Neo4j native ids (this mode is more suitable for using clone operation inside of complex queries as the Neo4j id have the guaranteed uniqueness)

In [13]:
print("Here we ignore pretty naming of new clones\n")
clone2_res = g.clone_node('a', ignore_naming=True)
print("Created clone by the name: ", clone2_res)
print("Properties of the resulting node: ")
print(g.get_node(clone2_res))
print("Properties of incident edges are also cloned, e.g: ")
print(g.get_edge(clone2_res, 'b'))

Here we ignore pretty naming of new clones

MATCH (x:node { id : 'a' })
WITH [] as sucIgnore, [] as predIgnore, x 
// match successors and out-edges of a node to be cloned
OPTIONAL MATCH (x)-[out_edge:edge]->(suc) 
WHERE NOT suc.id IS NULL AND NOT suc.id IN sucIgnore
WITH collect({neighbor: suc, edge: out_edge}) as suc_maps, predIgnore, x 
// match predecessors and in-edges of a node to be cloned
OPTIONAL MATCH (pred)-[in_edge:edge]->(x) 
WHERE NOT pred.id IS NULL AND NOT pred.id IN predIgnore
WITH collect({neighbor: pred, edge: in_edge}) as pred_maps, x, suc_maps 
// create a node corresponding to the clone
CREATE (new_node:node) 
WITH new_node, toString(id(new_node)) as uid, x.id as original_old, x, suc_maps, pred_maps 
// set the id property of the original node to NULL
SET x.id = NULL
// copy all the properties of the original node to the clone
SET new_node = x
// set id property of the clone to neo4j-generated id
SET new_node.id = toString(id(new_node)), new_node.count = NULL
// s

The third option is to specify the id of the new node manually

In [14]:
clone3_res = g.clone_node('a', 'a_clone')
print("Created clone by the name: ", clone3_res)
print("Properties of the resulting node: ")
print(g.get_node(clone3_res))
print("Properties of incident edges are also cloned, e.g: ")
print(g.get_edge(clone3_res, 'b'))

MATCH (x:node { id : 'a' })
WITH [] as sucIgnore, [] as predIgnore, x 
// match successors and out-edges of a node to be cloned
OPTIONAL MATCH (x)-[out_edge:edge]->(suc) 
WHERE NOT suc.id IS NULL AND NOT suc.id IN sucIgnore
WITH collect({neighbor: suc, edge: out_edge}) as suc_maps, predIgnore, x 
// match predecessors and in-edges of a node to be cloned
OPTIONAL MATCH (pred)-[in_edge:edge]->(x) 
WHERE NOT pred.id IS NULL AND NOT pred.id IN predIgnore
WITH collect({neighbor: pred, edge: in_edge}) as pred_maps, x, suc_maps 
// search for a node with the same id as the clone id
OPTIONAL MATCH (same_id_node:node { id : 'a_clone'}) 
WITH same_id_node,  CASE WHEN same_id_node IS NOT NULL THEN (coalesce(same_id_node.count, 0) + 1) ELSE 0 END AS same_id_node_new_count, x, suc_maps, pred_maps
// generate new id if the same id node was found
// and filter edges which will be removed 
WITH same_id_node, same_id_node_new_count, 'a_clone' + CASE WHEN same_id_node_new_count <> 0 THEN toString(same_i

In [15]:
merge1_res = g.merge_nodes1(["x", "f"])
print("Properties of the resulting node: {}".format(merge1_res))
print(g.get_node(merge1_res))

MATCH (x:node { id : 'x'}), (f:node { id : 'f'}) 
// accumulate all the attrs of the nodes to be merged
WITH [] as new_props, f, x
WITH new_props + REDUCE(pairs = [], k in keys(x) | 
	pairs + REDUCE(inner_pairs = [], v in x[k] | 
		inner_pairs + {key: k, value: v})) as new_props, f, x
WITH new_props + REDUCE(pairs = [], k in keys(f) | 
	pairs + REDUCE(inner_pairs = [], v in f[k] | 
		inner_pairs + {key: k, value: v})) as new_props, f, x
WITH apoc.map.groupByMulti(new_props, 'key') as new_props, f, x
WITH apoc.map.fromValues(REDUCE(pairs=[], k in keys(new_props) | 
	pairs + [k, REDUCE(values=[], v in new_props[k] | 
		values + CASE WHEN v.value IN values THEN [] ELSE v.value END)])) as new_props, f, x
SET x = new_props
WITH x as merged_node, f
// search for a node with the same id as the clone id
OPTIONAL MATCH (same_id_node:node { id : 'x_f'}) 
WITH same_id_node,  CASE WHEN same_id_node IS NOT NULL THEN (coalesce(same_id_node.count, 0) + 1) ELSE 0 END AS same_id_node_new_count, merged_

In [ ]:
merge2_res = g.merge_nodes(["a_b", "d"])
print("Properties of the resulting node: {}".format(merge2_res))
print(g.get_node(merge2_res))

## Rewriting Neo4j graph

### 1. Create an SqPO rewriting rule

In [ ]:
pattern = nx.DiGraph()
pattern.add_nodes_from(["x", "y", "z"])
pattern.add_edges_from([("y", "x"), ("y", "z"), ("z", "z")])

In [ ]:
rule = Rule.from_transform(pattern)
clone_name1, _ = rule.inject_clone_node("y")
clone_name2, _ = rule.inject_clone_node("y")
rule.inject_remove_edge(clone_name2, "z")
rule.inject_remove_node("x")
rule.inject_add_node("new_node")
rule.inject_add_edge("new_node", "z")

In [ ]:
plot_rule(rule)

In [ ]:
instances = g.find_matching(pattern)
print("Instances: ", instances)

In [ ]:
rhs_g = g.rewrite(rule, instances[0])
print("\n\nRewriting rule to commands: \n")
print(rule.to_commands())

In [ ]:
rhs_g